In [1]:
import torch
from torchtext import data

from tqdm import tqdm

import numpy as np
from scipy import stats

import pickle
import dill
import umsgpack
import time
import json

# Import the libraries
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

**Set constants**

In [2]:
DATASET_PATH = f"../.data/miguel"
SRC_LANG = "en"
TRG_LANG = "es"
SOS_WORD = '<sos>'
EOS_WORD = '<eos>'
MAX_SEQ_LENGTH = 100

## Load dataset

To speed things up, I prefer to use torchtext directly in order to read the CSV files, preprocess
them and tokenize each pair.

I'm gonna use the tokenizer from Spacy, which is a Natural Language Processing library that is blazingly fast, suitable
for large datasets, with support for many language and hundreds of features.

This step can take a while but since I plan to save our tokenized datasets, it  won't be a problem.

Also note that I'm converting everything to lowercase, and adding the `<sos>` and `<eos>` tokens to our pairs.

***Note:** Keep in mind that I share (by reference) the SRC/TRG fields between the train, dev and test partitions.*

In [3]:
SRC = data.Field(tokenize='spacy', tokenizer_language=SRC_LANG, init_token=SOS_WORD, eos_token=EOS_WORD, lower=True)
TRG = data.Field(tokenize='spacy', tokenizer_language=TRG_LANG, init_token=SOS_WORD, eos_token=EOS_WORD, lower=True)

/home/salvacarrion/.local/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/salvacarrion/.local/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
data_fields = [('src', SRC), ('trg', TRG)]  # Shared fields
train_data, dev_data, test_data = data.TabularDataset.splits(path=f'{DATASET_PATH}/preprocessed/',
                                                             train='train.csv', validation='dev.csv', test='test.csv',
                                                             format='csv', fields=data_fields, skip_header=True)

/home/salvacarrion/.local/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/home/salvacarrion/.local/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)


## Dataset exploration

On the first part of the exploration we perform some sanity checks to make sure that our preprocessed dataset are valid.

Print the number of pairs for each partition: train, dev and test.

In [ ]:
print("Total pairs:")
print(f"\t- Train: {len(train_data.examples)}")
print(f"\t- Dev: {len(dev_data.examples)}")
print(f"\t- Test: {len(test_data.examples)}")

### Viewing tokens

Now, we can the print the tokenized pairs to see what are we really doing. I usually like to print the first and last
`n` pairs (and a few random ones).


----
**Old:**
*Below we see a few things. First, the tokenization looks good. Nevertheless, we see that there are still a few
things that can be annoying such as `\xa0` or `"`. Maybe I should normalize the strings to avoid those problems.*

In [ ]:
def view_pairs(pairs, indices):
    for i, idx in enumerate(indices):
        (src, trg) = pairs[idx].src, pairs[idx].trg
        print(f"#{i+1}: " + "-"*20)
        print(f"src => {src}")
        print(f"trg => {trg}")
    print("")

In [ ]:
n=5
print("Head: " + "#"*20)
print("(Firsts) Train dataset: " + "*"*20)
view_pairs(train_data.examples, indices=range(0, n))

print("(Firsts) Dev dataset: " + "*"*20)
view_pairs(dev_data.examples, indices=range(0, n))

print("(Firsts) Test dataset: " + "*"*20)
view_pairs(test_data.examples, indices=range(0, n))

print("Tail: " + "#"*20)
print("(Lasts) Train dataset: " + "*"*20)
view_pairs(train_data.examples, indices=range(-1,-n-1,-1))

print("(Lasts) Dev dataset: " + "*"*20)
view_pairs(dev_data.examples, indices=range(-1,-n-1,-1))

print("(Lasts) Test dataset: " + "*"*20)
view_pairs(test_data.examples, indices=range(-1,-n-1,-1))


### Sequence lengths

Since some of models that use of attention have a quadratic growth with regard the length of a sequence, we need to 
carefully study the lengths of the pairs in our dataset so that we can choose the maximum length of a sequence in a 
educated way.

From the results below we observe a huge gap between pairs. This gap gets accentuated for the train and dev/test partitions. In the former, the gap is greater than in the later.

In [ ]:
# Get lengths
train_lenghts = np.array([(len(pair.src), len(pair.trg)) for pair in train_data.examples])
dev_lenghts = np.array([(len(pair.src), len(pair.trg)) for pair in dev_data.examples])
test_lenghts = np.array([(len(pair.src), len(pair.trg)) for pair in test_data.examples])

# Get min lengths
train_min_len = train_lenghts.min(axis=0).astype(int)
dev_min_len = dev_lenghts.min(axis=0).astype(int)
test_min_len = test_lenghts.min(axis=0).astype(int)


# Get max lengths
train_max_len = train_lenghts.max(axis=0).astype(int)
dev_max_len = dev_lenghts.max(axis=0).astype(int)
test_max_len = test_lenghts.max(axis=0).astype(int)

In [ ]:
print(f"Train => Range length: src={train_min_len[0]}-{train_max_len[0]} | trg={train_min_len[1]}-{train_max_len[1]}")
print(f"Dev => Range length: src={dev_min_len[0]}-{dev_max_len[0]} | trg={dev_min_len[1]}-{dev_max_len[1]}")
print(f"Test => Range length: src={test_min_len[0]}-{test_max_len[0]} | trg={test_min_len[1]}-{test_max_len[1]}")

#### Checking for zero-length sequences

Additionally, I like to check if there is any sequence with length zero. This is more important that it looks, since an empty sequence could indicate an error or problem in the dataset which could lead to a waste of computational resources, a `RuntimeError` in the middle of the training or even jeopardising the training of our model

In [ ]:
tr_no_sent = np.sum(train_lenghts == 0, axis=0).astype(int)
dev_no_sent = np.sum(dev_lenghts == 0, axis=0).astype(int)
ts_no_sent = np.sum(test_lenghts == 0, axis=0).astype(int)

print(f"Train => Sentences with no words: SRC={tr_no_sent[0]} | TRG={tr_no_sent[1]}")
print(f"Dev => Sentences with no words: SRC={dev_no_sent[0]} | TRG={dev_no_sent[1]}")
print(f"Test => Sentences with no words: SRC={ts_no_sent[0]} | TRG={ts_no_sent[1]}")

### Viewing extreme sequences

To know the range in which the sequences are ranging is important, but it is even more important to understand what are the extreme pairs in that range. By studying these extreme cases, we can educatively tune our model to get the most of our dataset without compromissing their requirements too much.

To get these extreme pairs, we simply keep the indices of the sorted lengths using `argsort`.

In [ ]:
def view_lengths(raw, indices):
    for i, idx in enumerate(indices):
        p1_src, p1_trg = raw[idx[0]].src, raw[idx[0]].trg  # Shortest src (+trg)
        p2_src, p2_trg = raw[idx[1]].src, raw[idx[1]].trg  # Shortest trg (+src)

        
        print(f"#{i+1}: " + "-"*20)
        print(f"[Shortest SRC: idx={idx[0]}]")
        print(f"\t- src (len={len(p1_src)}): {p1_src} => {p1_trg}")
        
        print(f"[Shortest TRG: idx={idx[1]}]")
        print(f"\t- trg (len={len(p2_src)}): {p2_src} => {p2_trg}")
    print("")

In [ ]:
# Sort columns independently (ascending)
train_sort_len_idx = np.argsort(train_lenghts, axis=0)
dev_sort_len_idx = np.argsort(dev_lenghts, axis=0)
test_sort_len_idx = np.argsort(test_lenghts, axis=0)

Now we see which sentences contain the minimum and the maximum number of tokens. The shortests sequences correspond to word-to-word translations such as: `italy => italia` or `Informality => Informalidad`.
 
**Notice** that columns in the array are sorted independely. This is because a source sequence can be a single word but its translation can be many (and viceversa), such as: `shopping => ir de compras`

In [ ]:
n=5
print("Shortest: " + "#"*20)
print("(Shortest) Train dataset: " + "*"*20)
view_lengths(train_data.examples, indices=train_sort_len_idx[:n])

print("(Shortest) Dev dataset: " + "*"*20)
view_lengths(dev_data.examples, indices=dev_sort_len_idx[:n])

print("(Shortest) Test dataset: " + "*"*20)
view_lengths(test_data.examples, indices=test_sort_len_idx[:n])

print("Longest: " + "#"*20)
print("(Longest) Train dataset: " + "*"*20)
view_lengths(train_data.examples, indices=train_sort_len_idx[::-1][:n])

print("(Longest) Dev dataset: " + "*"*20)
view_lengths(dev_data.examples, indices=dev_sort_len_idx[::-1][:n])

print("(Longest) Test dataset: " + "*"*20)
view_lengths(test_data.examples, indices=test_sort_len_idx[::-1][:n])



### Plotting length distribution

After printing a few examples of pairs, it is interesting to plot the length distribution of the different partitions so we can get an intuition on where to set the length cut.

From the plots below, we see that the train partition come from a different source than our dev and test partitions. Luckily, this won't be a problem for setting out cut since the training partition is way bigger than our dev/test partitions

In [ ]:
# Get lengths
tr_src_len = train_lenghts[:, 0]
tr_trg_len = train_lenghts[:, 1]
dev_src_len = dev_lenghts[:, 0]
dev_trg_len = dev_lenghts[:, 1]
ts_src_len = test_lenghts[:, 0]
ts_trg_len = test_lenghts[:, 1]

# Pair data and labels
h_data = [tr_src_len, tr_trg_len, dev_src_len, dev_trg_len, ts_src_len, ts_trg_len]
h_labels = ['tr_src_len', 'tr_trg_len', 'dev_src_len', 'dev_trg_len', 'ts_src_len', 'ts_trg_len']

In [ ]:
def get_hist(ax, data, labels, title, bins=25):
    # Draw the plot
    ax.hist(data, bins=bins, density=True, label=labels, linewidth=0.5)

    # Title and labels
    ax.set_title(title)
    ax.set_xlabel("Lengths")
    ax.set_ylabel("Frenquency (normed)")
    ax.legend()


plt.rcParams['figure.figsize'] = [10, 10]  # Fix size
plt.rcParams['figure.dpi'] = 300  # Fix white bars (+linewidth=0.5)

fig, axs = plt.subplots(4)
get_hist(axs[0], h_data, h_labels, f"Histogram of Train+Dev+Test")
get_hist(axs[1], h_data[0:2], h_labels[0:2], f"Histogram of Train")
get_hist(axs[2], h_data[2:4], h_labels[2:4], f"Histogram of Dev")
get_hist(axs[3], h_data[4:6], h_labels[4:6], f"Histogram of Test")

plt.rcParams['figure.figsize'] = [10, 10]
plt.subplots_adjust(hspace=1.0)

### Sequence length stats

We can also get some statistical values from partitions to get a rough idea of how long is the average sequence or in which range of values we can find 95% of our data.

By doing this, we are obtaining a value with which perform the sequence cut in the dataset.

In [ ]:
# Get means
train_mean_len = train_lenghts.mean(axis=0).round().astype(int)
dev_mean_len = dev_lenghts.mean(axis=0).round().astype(int)
test_mean_len = test_lenghts.mean(axis=0).round().astype(int)

print(f"Train => Mean length: src={train_mean_len[0]} | trg={train_mean_len[1]}")
print(f"Dev => Mean length: src={dev_mean_len[0]} | trg={dev_mean_len[1]}")
print(f"Test => Mean length: src={test_mean_len[0]} | trg={test_mean_len[1]}")

### Setting the length threshold

Let's say that our model has only memory for sequences with a maximum of 150 tokens. With this value in mind (and the sequence length distribution), we can now exactly know how much data we are leaving behind. 

This is important so that we can get the most of our model by carefully tuning it with the statistical values that we have calculated. Additionally, since this is an educated process, we will also have a rough idea of how much our model could improve if we increase the computational resources.


In [ ]:
tr_src_p_score = stats.percentileofscore(train_lenghts[:, 0], MAX_SEQ_LENGTH)
tr_trg_p_score = stats.percentileofscore(train_lenghts[:, 1], MAX_SEQ_LENGTH)
print(f"Train: percentile of {MAX_SEQ_LENGTH}: src={tr_src_p_score} | trg={tr_trg_p_score}")

dev_src_p_score = stats.percentileofscore(dev_lenghts[:, 0], MAX_SEQ_LENGTH)
dev_trg_p_score = stats.percentileofscore(dev_lenghts[:, 1], MAX_SEQ_LENGTH)
print(f"Train: percentile of {MAX_SEQ_LENGTH}: src={dev_src_p_score} | trg={dev_trg_p_score}")

ts_src_p_score = stats.percentileofscore(test_lenghts[:, 0], MAX_SEQ_LENGTH)
ts_trg_p_score = stats.percentileofscore(test_lenghts[:, 1], MAX_SEQ_LENGTH)
print(f"Train: percentile of {MAX_SEQ_LENGTH}: src={ts_src_p_score} | trg={ts_trg_p_score}")


In [ ]:
def get_percentiles(src_trg_lengths, lower_q, upper_q):
    p = np.percentile(src_trg_lengths,  [lower_q, upper_q], axis=0)
    (lb_tr_src_len, lb_tr_trg_len), (ub_tr_src_len, ub_tr_trg_len) = p

    print(f"- SRC ({lower_q}-{upper_q}%): {int(lb_tr_src_len)}-{int(ub_tr_src_len)}")
    print(f"- TRG ({lower_q}-{upper_q}%): {int(lb_tr_trg_len)}-{int(ub_tr_trg_len)}")

    return p

lower_q, upper_q = (0, 99.65)  # Percentage
print("Percentile Train:")
tr_p = get_percentiles(train_lenghts, *(lower_q, upper_q))
print("Percentile Dev:")
dev_p = get_percentiles(dev_lenghts, *(lower_q, upper_q))
print("Percentile Test:")
ts_p = get_percentiles(test_lenghts, *(lower_q, upper_q))


### Removing pairs

NExt, we remove all pairs where either the *source* or *target* exceeds certain threshold.

As we are deleting elements from a list while iterating, we need to start from the end.
There are many ways to do this that are faster (e.g.: list comprehesion, creating a new copy,...), I have
used this method because it simply works.

In [ ]:
def remove_max_length(data, max_length):
    removed = 0
    for i in tqdm(reversed(range(len(data.examples))), total=len(data.examples)):
        if len(data.examples[i].src) > max_length or len(data.examples[i].trg) > max_length:
            del data.examples[i]
            removed += 1
    return removed

In [ ]:
print(f"Train pairs (before): {len(train_data)}")
tr_removed = remove_max_length(train_data, MAX_SEQ_LENGTH)
print(f"Train pairs (now): {len(train_data)}")
print(f"Train pairs (removed): {tr_removed}")
print("")
print(f"Dev pairs (before): {len(dev_data)}")
print(f"Dev pairs (now): {len(dev_data)}")
dev_removed = remove_max_length(dev_data, MAX_SEQ_LENGTH)
print(f"Dev pairs (removed): {dev_removed}")
print("")
print(f"Test pairs (before): {len(test_data)}")
print(f"Test pairs (now): {len(test_data)}")
ts_removed = remove_max_length(test_data, MAX_SEQ_LENGTH)
print(f"Test pairs (removed): {ts_removed}")

### Vocabulary

For the next step we will build the vocabulary. That is, creating a list of unique words/tokens that our model will
"understand". All the other words that are not in the vocabulary will be marked as `<unk>`.

Technically, we cannot use the dev partition for anything but evaluating our model during training. We do this in
order to avoid overfitting our model on the "super secret" test partition. Hance, we have to build the vocabulary
using only the train partition.



In [ ]:
# Define Train fields (they're already defined above)
# SRC = data.Field(tokenize='spacy', tokenizer_language=SRC_LANG, init_token=SOS_WORD, eos_token=EOS_WORD, lower=True)
# TRG = data.Field(tokenize='spacy', tokenizer_language=TRG_LANG, init_token=SOS_WORD, eos_token=EOS_WORD, lower=True)

# Build vocabulary only with the training data
SRC.build_vocab(train_data.src)
TRG.build_vocab(train_data.trg)

#### Vocabulary size

Once the vocabulary is built, we need to check its size and see if we need to perform some refinement.

Anything between 10k to 15k seems reasonable. We could go for even 30k for big models.

In [ ]:
print(f"Words in SRC vocabulary: {len(SRC.vocab.freqs)}")
print(f"Words in TRG vocabulary: {len(TRG.vocab.freqs)}")

#### Vocabulary refinement

We can use all unique tokens found in the train partition as our "vocabulary", but this is usually not a good idea since
many of the less frequent words in the vocabulary will be typos, personal names, words in another languages, etc.
Moreover, using everything as our vocabulary would notable increase the resources needed.

Luckily, there is a really simple solution to deal with this problem: Using the `k` most frequent words.

From the words below, we see that everything is reasonable as since the vocabulary is not that big, we could leave
it as is.

In [ ]:
n=50
print("Train vocab:")
print("=> SRC (most common): " + str(SRC.vocab.freqs.most_common(n)))
print("=> SRC (least common): " + str(SRC.vocab.freqs.most_common()[-n-1:]))
print("=> TRG (most common): " + str(TRG.vocab.freqs.most_common(n)))
print("=> TRG (least common): " + str(TRG.vocab.freqs.most_common()[-n-1:]))

## Saving preprocessed dataset

Since preprocessing and tokenizing a big dataset can take time, I recommend to save it always, if possible, in other
to save us some time when debugging our model.

For torchtext, we'll need to save the preprocessed examples for the train, dev and test partitions, along with their
fields. Depending on the case we can ignore the last part. Here, I'll save only the train fields with their
vocabularies.

**Note:** *To decide the serialization approach I had to do some test as the naive pickle-version either doesn't work
or is too slow.*

In [ ]:
def save_examples(dataset, savepath):
    start = time.time()

    total = len(dataset.examples)
    with open(savepath, 'w') as f:
        # Save num. elements
        f.write(json.dumps(total))
        f.write("\n")

        # Save elements
        for pair in tqdm(dataset.examples, total=total):
            data = [pair.src, pair.trg]
            f.write(json.dumps(data))
            f.write("\n")

    end = time.time()
    print(end - start)

save_examples(train_data, f"{DATASET_PATH}/tokenized/train.json")
save_examples(dev_data, f"{DATASET_PATH}/tokenized/dev.json")
save_examples(test_data, f"{DATASET_PATH}/tokenized/test.json")
print("Tokenized examples saved!")


## Loading preprocessed dataset

Similarly, we can also load the preprocessed datasets.

In [ ]:

def load_examples(filename):
    start = time.time()

    examples = []
    with open(filename, 'rb') as f:
        # Read num. elements
        line = f.readline()
        total = json.loads(line)

        # Save elements
        for i in tqdm(range(total), total=total):
            line = f.readline()
            example = json.loads(line)
            examples.append(example)

    end = time.time()
    print(end - start)
    return examples


train_data = load_examples(f"{DATASET_PATH}/tokenized/train.json")
dev_data = load_examples(f"{DATASET_PATH}/tokenized/dev.json")
test_data = load_examples(f"{DATASET_PATH}/tokenized/test.json")
print("Tokenized examples loaded!")



In [ ]:
print("Total pairs:")
print(f"\t- Train: {len(train_data.examples)}")
print(f"\t- Dev: {len(dev_data.examples)}")
print(f"\t- Test: {len(test_data.examples)}")